In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
import subprocess
import shutil
import time
import pandas as pd
import pymongo as mg

In [2]:
# MongoDB 연결
client = mg.MongoClient(host='mongodb://localhost:27017')
database = client['project_nutrients']
collection = database['11st_corelsterolsupplement_review']

In [3]:
# Chrome 웹 드라이버 설정
subprocess.Popen(r'C:\Program Files\Google\Chrome\Application\chrome.exe --remote-debugging-port=9222 --user-data-dir="C:\chrometemp"')
options = webdriver.ChromeOptions()
options.add_experimental_option("debuggerAddress", "127.0.0.1:9222")

In [4]:
# open chrome browser
browser = webdriver.Chrome(executable_path='C:/Users/04-02/Develops/chromedriver.exe', options=options)
browser.set_window_size(1560,2000)

In [5]:
# url in address window 리뷰 많은 순/ 영양제별 검색어 변경하기  
browser.get('https://search.11st.co.kr/pc/total-search?kwd=%25EC%25BD%259C%25EB%25A0%2588%25EC%258A%25A4%25ED%2585%258C%25EB%25A1%25A4%2520%25EC%2598%2581%25EC%2596%2591%25EC%25A0%259C&fromACK=recent&tabId=TOTAL_SEARCH&sortCd=I&pageNo=1')
browser.implicitly_wait(10)

In [6]:
# pagination 하기
## 리뷰건수 확인 
product_total = browser.find_element_by_css_selector('div.search_content > div > p > span')
product_total_str = product_total.text
product_total_del = product_total_str.replace(',', '')
product_total_count = int(product_total_del)

In [7]:
print(product_total_count) 

# 한 페이지 상품 목록 수
product_standard_count_per = 60 

6291


In [8]:
# 상품 총 페이지수
loop_count_int = int(product_total_count / product_standard_count_per) 
print(loop_count_int)  

104


In [ ]:
current_page = 1
# pagination 버튼을 끝까지 순환하면서 페이지 이동합니다.
while current_page <= loop_count_int:
    for i in range(1, 61):  # 1부터 60까지 순회합니다.
        try:
            product_page = f'#section_commonPrd > div.c-search-list > ul > li:nth-child({i}) > div > a'
            product = browser.find_element_by_css_selector(product_page)
            product.click()
            # 새로 열린 창으로 브라우저 컨텍스트를 전환합니다.
            browser.switch_to.window(browser.window_handles[-1]) 
            browser.implicitly_wait(2)
          
            
            try:
                product_name = browser.find_element_by_css_selector('#layBodyWrap > div > div.s_product.s_product_detail > div.l_product_cont_wrap > div > div.l_product_view_wrap > div.l_product_summary > div.l_product_side_info > div.c_product_info_title > h1').text
            except :
                product_name = str()

            # product new window / produt name 옆 리뷰 보기클릭->페이지 하단 리뷰로 이동
            browser.find_element_by_css_selector('#prdReview > strong').click() 
            # iframe을 html로 전환시킴. ifram에 name은 'ifrmReview'
            browser.switch_to.frame('ifrmReview')
         
            # 전체 리뷰 펼치기: 리뷰 더보기 button click
            while True:
                try:
                    button = WebDriverWait(browser, 10).until(
                        EC.element_to_be_clickable((By.CSS_SELECTOR, '#review-list-page-area > div > button'))
                    )
                    button.click()
                    time.sleep(2)  # 사이트 로딩 대기
                except Exception as e:
                    print(f'예외 발생: {e}')
                    break  # 예외 발생 시 루프를 종료

            # 단일 리뷰 펼치기 : 더보기 button click
            review_bundle = browser.find_elements_by_css_selector('.review_list_element')
            len(review_bundle)

            ul_index = 1
            li_index = 1

            for i in range(1, len(review_bundle)+1):
                try:
                    time.sleep(1)
                    expand_button_css_selector = f'#review-list-page-area > ul:nth-child({ul_index}) > li:nth-child({li_index}) > div > div > div.cont_text_wrap > p.cont_btn.review-expand > button.c_product_btn.c_product_btn_more6.review-expand-open-text' 
                    expand_button = browser.find_elements_by_css_selector(expand_button_css_selector)

                    expand_button[0].click()  # Click the button if it exists 
                except:
                    time.sleep(1)
                    pass
                    # print('pass')  
                if li_index == 10:
                    ul_index += 1
                    li_index = 1
                else:
                    li_index += 1 

            # 리뷰들 리스트 (제품명,작성자,날짜,별점,내용)< 제품명포함 위에 있어서 같이 가져올 수 있음.
            reviews_list = list()
            def find_total_reviews():
                review_elements = browser.find_elements_by_css_selector('#review-list-page-area dl')
                total_reviews = len(review_elements)
                return total_reviews

            # "사용 가능한 리뷰의 총 수 찾기"
            total_reviews = find_total_reviews()

            # "ul"과 "li" 요소의 인덱스를 초기화하는 변수 설정
            ul_index = 1
            li_index = 1

            # "리뷰의 총 수까지 1부터 루프 실행"
            for i in range(1, total_reviews + 1):
                # 계산된 인덱스로 CSS 선택자 구성 
                css_selector_writer = f'#review-list-page-area > ul:nth-child({ul_index}) > li:nth-child({li_index}) > dl > dt'
                css_selector_date = f'#review-list-page-area > ul:nth-child({ul_index}) > li:nth-child({li_index}) > div > p.side > span'
                css_selector_star = f'#review-list-page-area > ul:nth-child({ul_index}) > li:nth-child({li_index}) > div > p.grade > span > em'
                css_selector_content = f'#review-list-page-area > ul:nth-child({ul_index}) > li:nth-child({li_index}) > div > div'

                try:
                    try:
                        review_writer = browser.find_element_by_css_selector(css_selector_writer).text
                    except :
                        review_writer = str()
                    try:    
                        review_date = browser.find_element_by_css_selector(css_selector_date).text
                    except :
                        review_date = str()
                    try:      
                        review_star = browser.find_element_by_css_selector(css_selector_star).text
                    except :
                        review_star = str()
                    try:
                        review_content = browser.find_element_by_css_selector(css_selector_content).text
                    except :
                        review_content = str()
                    # 리뷰 정보를 딕셔너리로 저장하고 리스트에 추가
                    review_list = [product_name, review_writer, review_date, review_star, review_content]
                    reviews_list.append(review_list)
                except:
                    break  

                # 패턴을 기반으로 ul과 li 인덱스를 업데이트
                if li_index == 10:
                    ul_index += 1
                    li_index = 1
                else:
                    li_index += 1


            review_columns_name = ["product_name", "review_writer", "review_date", "review_star", "review_content"]
            df_reviews = pd.DataFrame(data=reviews_list, columns=review_columns_name)
            data_dict = df_reviews.to_dict(orient='records')
            collection.insert_many(data_dict)
            print(len(reviews_list))
            
            # 현재 페이지를 닫습니다.
            browser.close()
            # 다음 상품을 클릭하기 전에 원래의 창으로 다시 전환합니다.
            browser.switch_to.window(browser.window_handles[0])
                        
        except:
            print(f'에러: {i}')
            pass
    # Pagination 버튼을 클릭합니다.
    # 다음 페이지 버튼이 클릭 가능할 때까지 대기 
    current_page += 1
    wait = WebDriverWait(browser, 3)  
    page_button_css = f'#section_commonPrd > nav > ul > li:nth-child({current_page % 10 + 2}) > button'
    page_button = browser.find_element_by_css_selector(page_button_css)
    page_button.click()
    
    # 10번째 페이지일 경우 next 버튼 클릭
    if current_page % 10 == 0:
        next_button = browser.find_element_by_css_selector('#section_commonPrd > nav > ul > li.next > button')
        next_button.click()


예외 발생: Message: 

